In [4]:
from os import listdir
from os.path import isfile, join
import os
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import kagglehub

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Activation, MaxPooling1D, Dropout
from tensorflow.keras.utils import plot_model,to_categorical

/Users/nikisingh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nikisingh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Download latest version of the database 
path = kagglehub.dataset_download("vbookshelf/respiratory-sound-database")

print("Path to dataset files:", path)

100%|██████████| 3.69G/3.69G [03:59<00:00, 16.5MB/s]

Extracting files...


Path to dataset files: /Users/nikisingh/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2


In [6]:
# extract lables for kaggle dataset
# Path to patient diagnosis CSV
csv_path_kaggle = os.path.join(path, "Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv")

# Load CSV
df_kaggle = pd.read_csv(csv_path_kaggle, header=None, names=["Patient_ID", "Diagnosis"])

# Convert to dictionary {Patient_ID: Diagnosis}
patient_labels_kaggle = dict(zip(df_kaggle["Patient_ID"].astype(str), df_kaggle["Diagnosis"]))

# Display first few mappings
print(list(patient_labels_kaggle.items())[:10])  # Check some values

[('101', 'URTI'), ('102', 'Healthy'), ('103', 'Asthma'), ('104', 'COPD'), ('105', 'URTI'), ('106', 'COPD'), ('107', 'COPD'), ('108', 'LRTI'), ('109', 'COPD'), ('110', 'COPD')]


In [9]:
df_kaggle
#patient_labels_kaggle

,Patient_ID,Diagnosis
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI
...,...,...
121,222,COPD
122,223,COPD
123,224,Healthy
124,225,Healthy


In [10]:
# extract labels for Mendeley dataset
csv_path_mendeley = "audioDataMendeley/Data annotation.csv"

# read csv and only take necessary columns
df_mendeley_raw = pd.read_csv(csv_path_mendeley)
col_index = df_mendeley_raw.columns.get_loc('Diagnosis')
df_mendeley = df_mendeley_raw.iloc[:, :col_index + 1]

# drop heart failure diagnosis
df_mendeley = df_mendeley[~df_mendeley["Diagnosis"].str.lower().str.contains("heart failure", na=False)]

# Change N (normal) to Healthy
df_mendeley["Diagnosis"] = df_mendeley["Diagnosis"].replace({"N": "Healthy"})

# add patient ID column and set as index
df_mendeley.insert(0, "Patient_ID", range(1, len(df_mendeley) + 1))

# formatting for capitalizig diasnosis with expection of acronyms
keep_upper = {"COPD", "BRON"}

def formatting(diagnosis):
    if pd.isna(diagnosis):
        return diagnosis
    text_clean = diagnosis.strip()
    # If the diagnosis (case-insensitive) matches one we want to keep uppercase
    if text_clean.upper() in keep_upper:
        return text_clean.upper()
    else:
        return text_clean.title()

df_mendeley["Diagnosis"] = df_mendeley["Diagnosis"].apply(formatting)

# Convert to dictionary {Patient_ID: Diagnosis}
patient_labels_mendeley = dict(zip(df_mendeley["Patient_ID"].astype(str), df_mendeley["Diagnosis"]))

print(list(patient_labels_mendeley.items())[:10])  # Check some values

[('1', 'Asthma'), ('2', 'Asthma'), ('3', 'Asthma'), ('4', 'Plueral Effusion'), ('5', 'Plueral Effusion'), ('6', 'Asthma'), ('7', 'Asthma'), ('8', 'Asthma'), ('9', 'Asthma'), ('10', 'COPD')]


In [12]:
df_mendeley


,Patient_ID,Age,Gender,Location,Sound type,Diagnosis
0,1,70,M,P L L,I E W,Asthma
1,2,52,F,P L L,E W,Asthma
2,3,50,F,P L L,I E W,Asthma
5,4,81,M,P R L,I C B,Plueral Effusion
7,5,70,M,P L L,I C B,Plueral Effusion
...,...,...,...,...,...,...
107,87,63,M,P R L,E W,COPD
108,88,26,M,P L M,N,Healthy
109,89,62,M,P L L,E W,COPD
110,90,51,M,P R L,E W,COPD
